In [1]:
import zipfile
import os
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
from sklearn.model_selection import train_test_split
from efficientnet_pytorch import EfficientNet

# Step 1: Load the train and test datasets
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

# Step 2: Custom Dataset Class to Load Images
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = 1 if self.dataframe.iloc[idx, 1] == 'healthy' else 0
        image = datasets.folder.default_loader(img_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label

# Step 3: Define transformations (for training and testing)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet expects 224x224 input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

# Step 4: Create DataLoaders
train_dataset = CustomImageDataset(dataframe=train_df, transform=train_transforms)
test_dataset = CustomImageDataset(dataframe=test_df, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Recreate the model architecture
modelOne = EfficientNet.from_pretrained('efficientnet-b0')
modelOne._fc = nn.Linear(in_features=modelOne._fc.in_features, out_features=2)  # Modify for binary classification

# Load the saved state_dict
modelOne.load_state_dict(torch.load('efficientnet_model.pth'))
modelOne = modelOne.to(device)  # Make sure to move the model to the correct device (CPU or GPU)
print("Model loaded successfully!")


Loaded pretrained weights for efficientnet-b0
Model loaded successfully!


In [5]:
modelResnet = models.resnet50(pretrained=False)

# Modify the final layer for binary classification (2 classes: healthy, unhealthy)
modelResnet.fc = torch.nn.Linear(in_features=modelResnet.fc.in_features, out_features=2)

# Step 2: Load the saved state_dict (weights), allowing for mismatch in final layer
modelResnet.load_state_dict(torch.load('resnet50_model.pth'), strict=False)
print("loaded successfully")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


loaded successfully


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score
import numpy as np
import torch

# Set both models to evaluation mode
modelResnet.eval()
modelOne.eval()

# Lists to store true labels, predictions, and predicted probabilities for ROC AUC and PR AUC
y_true = []
y_pred = []
y_prob = []  # To store predicted probabilities for ROC AUC and PR AUC

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Get predictions from ResNet
        outputs_resnet = modelResnet(images)
        probs_resnet = torch.softmax(outputs_resnet, dim=1)
        
        # Get predictions from EfficientNet
        outputs_efficientnet = modelOne(images)
        probs_efficientnet = torch.softmax(outputs_efficientnet, dim=1)
        
        # Average the probabilities
        avg_probs = (probs_resnet + probs_efficientnet) / 2
        
        # Get the final prediction (class with the highest probability)
        _, predicted = torch.max(avg_probs, 1)

        # Store the true labels, predictions, and probabilities for ROC AUC and PR AUC
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        
        # Store probabilities for the 'healthy' class (class 1)
        y_prob.extend(avg_probs[:, 1].cpu().numpy())

# Convert to numpy arrays for easier handling with sklearn
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

# Accuracy
accuracy = np.sum(y_pred == y_true) / len(y_true)

# Precision, Recall, F1 Score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Classification Report
class_report = classification_report(y_true, y_pred)

# ROC AUC (Area Under the ROC Curve)
roc_auc = roc_auc_score(y_true, y_prob)

# PR AUC (Area Under the Precision-Recall Curve)
pr_auc = average_precision_score(y_true, y_prob)

# Print the results
print(f'Ensemble Accuracy: {accuracy:.4f}')
print(f'Ensemble Precision: {precision:.4f}')
print(f'Ensemble Recall: {recall:.4f}')
print(f'Ensemble F1 Score: {f1:.4f}')
print(f'Ensemble ROC AUC: {roc_auc:.4f}')
print(f'Ensemble PR AUC: {pr_auc:.4f}')
print(f'Ensemble Confusion Matrix:\n{conf_matrix}')
print(f'Ensemble Classification Report:\n{class_report}')


Ensemble Accuracy: 0.9722
Ensemble Precision: 0.9708
Ensemble Recall: 0.9940
Ensemble F1 Score: 0.9822
Ensemble ROC AUC: 0.9935
Ensemble PR AUC: 0.9979
Ensemble Confusion Matrix:
[[ 44   5]
 [  1 166]]
Ensemble Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.90      0.94        49
           1       0.97      0.99      0.98       167

    accuracy                           0.97       216
   macro avg       0.97      0.95      0.96       216
weighted avg       0.97      0.97      0.97       216

